In [1]:
import os
import base64
from PIL import Image
from PIL import ImageFont
from PIL import ImageDraw 

In [2]:
temp_thumb_file = 'thumbnail_tmp.jpg'
X = 154
Y = 20
OFF = 20
small_size = (146,196)
thumb_size = (350, 200)
pic_offset = (2,2)
text_color = (0,0,0)
background_color = (255, 255, 255, 255)
font_size = 10
font_file = "Roboto-Regular.ttf"

In [3]:
def load_vcards(lines) : # form list of lines for each vcard, begin line excluded
    all_cards = list()
    card_lines = list()
    inside_card = False
    
    for line in lines:
        if ":" in line:  # parameter line 
            param, value = line.strip().split(':')
            if ("BEGIN" in param ):
                if not inside_card :
                    inside_card = True
                    #card_lines.append(line) 
                else :
                    print("invalid vcard format: END line missing")
                    return None
            elif ("END" in param) :
                if inside_card :
                    card_lines.append(line)
                    all_cards.append(card_lines)
                    card_lines = list()
                    inside_card = False
                else :
                    print("invalid vcard format: END line before BEGIN line")
                    return None
            else:
                card_lines.append(line)
        elif inside_card:
            card_lines.append(line)
        else :
            continue
    
    return all_cards

In [4]:
def decode_data(index, lines, pars, value) :
   
    photo_data = value
    
    for line in lines[index+1:] :
        if "END:VCARD" in line :
            break
        else :
            photo_data += line.strip()
    
    # we ignore parameters of encoding in pars list
    
    imgdata = base64.b64decode(photo_data)
    with open(temp_thumb_file, 'wb') as f:
        f.write(imgdata)
        
    return temp_thumb_file

In [5]:
def get_photo(card_lines) :
    
    photo_file = ''
    
    for index, line in enumerate(card_lines): # process others parameters
        if ":" in line:  # parameter line 
            param, value = line.strip().split(':')
            p, *p_var = param.split(';')
            if p == 'PHOTO' :
                if 'http' in value.lower():
                    # download photo from inet
                     break
                else:
                    # data is in the list, so we have to decode them
                    photo_file = decode_data(index, card_lines, p_var, value)
                    break
     
    return photo_file

In [6]:
def create_thumbnail(params, card_data) :

    background = Image.new('RGBA', thumb_size , background_color ) #current size is 350x200
    draw = ImageDraw.Draw(background)
    font = ImageFont.truetype(font_file, font_size) # font = ImageFont.truetype(<font-file>, <font-size>)
    
    if ('PHOTO' in params) and (card_data['PHOTO'] != '') :
        img = Image.open(card_data['PHOTO'], 'r')
        small_img = img.resize(small_size)
        background.paste(small_img, pic_offset)
        
    offset = Y
    for par in params:
        if par != 'PHOTO' :
            draw.text((X, offset), par+':  '+card_data[par], text_color, font=font)
            offset += OFF
           
    background.save( ( card_data['FN']+'.png').replace(' ', '_') )
   

In [7]:
def card_to_thumbnail(card_lines, params=['FN', 'TITLE', 'ORG', 'ADR',  'TEL', 'EMAIL', 'URL', 'PHOTO']): 
    
    card_data = dict()
   
    if 'PHOTO' in params :  # process PHOTO parameter if present
        card_data['PHOTO'] = result = get_photo(card_lines)
        if result == '' :
            print ("unable to load PHOTO data, parameter ignored")
    
    for line in card_lines: # process others parameters
        if ":" in line:  # parameter line 
            param, value = line.strip().split(':')
            p1, *_ = param.split(';')
            if (p1 in params) and (p1 not in card_data) :
                card_data[p1], *_ = value.strip().split(";")
                # card_data[p1], *_ = " ".join(value.split(';')).strip()
                       
    create_thumbnail(params, card_data)
    

In [8]:
def main() :
    file = "vcard_test.vcf"
    with open(file, encoding="utf8") as f:  
        data = f.readlines()
        
    list_of_cards = load_vcards(data)
    if (list_of_cards) :
        for card in list_of_cards :
            card_to_thumbnail(card)
        print("loaded {} vcards".format(len(list_of_cards)) )
    
    os.remove(temp_thumb_file)   # remove temporal jpg file

In [9]:
if __name__ == '__main__':
    main()

loaded 3 vcards
